In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
def parse_h2(file):
    h2 = np.nan
    se = np.nan
    with open(file) as file:
        for line in file:
            if "h2g (1,1):" in line:
                h2 = float(line.split()[2])
                se = float(line.split()[3][1:-1])
    return(np.array([h2, se]))

In [3]:
# need to keep this outside of a function
data = pd.read_csv("/Users/arun/Documents/pricelab/ps_gxe/newE/337K-pheno-cov-newE-dietpc.tab", sep="\t")

In [4]:
def get_bin_prev(trait, E_var, bin, nbins, data):
    if E_var == 'cov_SEX':
        iid_to_keep = data.loc[data['cov_SEX'] == int(bin)]['IID']
    else:
        iid_to_keep = pd.read_csv("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/bins_newE/337K_"+str(nbins)+"bin_"+E_var+"_"+str(bin), header=None)[0]
    prevalence = data[data['IID'].isin(iid_to_keep)][trait].mean()
    return(prevalence)

In [5]:
def obs2liab(h2, se, trait, E_var, b, nbins, data):
    prev = get_bin_prev(trait, E_var, b, nbins, data)
    T= norm.ppf(1-prev)
    z = np.exp(-T*T/2) / np.sqrt(2*3.1415927)
    h2l = h2*(prev*(1-prev)/np.square(z))
    sel = se**2*(prev*(1-prev)/np.square(z))**2
    return(h2l, sel)

In [6]:
def get_h2s(trait, E_var, nbins, data):
    newE_5bin = ['cov_ParticulateMatterAirPollution_irnt', 'cov_DIET']
    oldE_5bin = ['cov_ALCOHOL_DESC_ORDER', 'cov_PHYS2_DAYS_VERY_ACTIVE', 'cov_TOWNSEND_DEPRIVATION',
             'other_NAP_TIME', 'other_NO_WHEAT', 'other_TIME_TV']
    if nbins==5:
        if E_var in newE_5bin:
            dir='h2_newE_5bin/'
        elif E_var in oldE_5bin:
            dir='reml_337K/'
        else:
            return({'h2':[np.nan] * nbins,
                'se':[np.nan] * nbins})
    elif nbins==2:
        if E_var == "cov_SEX":
            dir='reml_337K_sex2/'
        else:
            dir='h2_newE/'
    else:
        return({'h2':[np.nan] * nbins,
                'se':[np.nan] * nbins})

    h2s = [np.nan] * nbins
    ses = [np.nan] * nbins
    for b in range(0,nbins):
        bin=str(b)
        if E_var == 'cov_SEX':
            file = "/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/"+dir+trait+"_"+bin+".log"
        else:
            file="/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/"+dir+trait+"_"+E_var+"_"+bin+".log"
        try:
            out = parse_h2(file)
            if 'disease' in trait:
                curr_h2, curr_se = obs2liab(out[0], out[1], trait, E_var, bin, nbins, data)
                h2s[b] = curr_h2
                ses[b] = curr_se   
            else:
                h2s[b] = out[0]
                ses[b] = out[1]
        except FileNotFoundError:
            continue
    return({'h2':h2s, "se":ses})

In [7]:
def get_Z(d):
    return((d['h2'][-1] - d['h2'][0]) / (np.sqrt(d['se'][-1]**2 + d['se'][0]**2)))

In [8]:
traits = list(pd.read_csv("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/data_update/traits.txt", header=None)[0])
E_vars = list(pd.read_csv("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/data_update/E.txt", header=None)[0])

# Output Z scores for 5 bin and 2 bin tests

In [10]:
with open("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/parsed/h2_Z-scores.csv", 'w') as f:
    for t in traits:
        for e in E_vars:
            print(t, e, get_Z(get_h2s(t, e, 5, data)), get_Z(get_h2s(t, e, 2, data)), sep=',', file=f)



In [14]:
with open("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/parsed/h2_values.csv", 'w') as f:
    for t in traits:
        for e in E_vars:
            d5 = get_h2s(t, e, 5, data)
            d2 = get_h2s(t, e, 2, data)
            print(t, e, *d5['h2'], *d5['se'], *d2['h2'], *d2['se'], sep=',', file=f)



In [11]:
with open("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/parsed/sex_h2_Z-scores.csv", 'w') as f:
    for t in traits:
        e='cov_SEX'
        print(t, e, get_Z(get_h2s(t, e, 2, data)), sep=',', file=f)

In [16]:
with open("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/parsed/sex_h2_values.csv", 'w') as f:
    for t in traits:
        e='cov_SEX'
        d2 = get_h2s(t, e, 2, data)
        print(t, e, *d2['h2'], *d2['se'], sep=',', file=f)

In [17]:
b=[5,5,2]
for jdx,E in enumerate(["cov_ALCOHOL_DESC_ORDER", "cov_TOWNSEND_DEPRIVATION", 'cov_SMOKING_STATUS']):
    for i in range(0,b[jdx]):
        print("body_BMIz", E, i, get_bin_prev("body_BMIz", E, i, b[jdx], data))

body_BMIz cov_ALCOHOL_DESC_ORDER 0 -0.17109267763460997
body_BMIz cov_ALCOHOL_DESC_ORDER 1 -0.1019906368432786
body_BMIz cov_ALCOHOL_DESC_ORDER 2 -0.01227704379616282
body_BMIz cov_ALCOHOL_DESC_ORDER 3 0.07243687371770093
body_BMIz cov_ALCOHOL_DESC_ORDER 4 0.16940570874944105
body_BMIz cov_TOWNSEND_DEPRIVATION 0 -0.09813356714022468
body_BMIz cov_TOWNSEND_DEPRIVATION 1 -0.05623489713402307
body_BMIz cov_TOWNSEND_DEPRIVATION 2 -0.01979241914849305
body_BMIz cov_TOWNSEND_DEPRIVATION 3 0.017242897748411392
body_BMIz cov_TOWNSEND_DEPRIVATION 4 0.11279494252188045
body_BMIz cov_SMOKING_STATUS 0 -0.06611222013144617
body_BMIz cov_SMOKING_STATUS 1 0.05835944873338994


In [18]:
for i in range(0,5):
    print(get_bin_prev("disease_T2D", "cov_ALCOHOL_DESC_ORDER", i, 5, data))

0.03155850701135454
0.02943923155601014
0.03640621989001053
0.04245415869910022
0.06864707015905486


In [19]:
print(get_bin_prev("blood_WHITE_COUNT", 'cov_SMOKING_STATUS', 0, 2, data))
print(get_bin_prev("blood_WHITE_COUNT", 'cov_SMOKING_STATUS', 1, 2, data))

-0.10671002331335461
0.13556158804177973


In [20]:
for i in range(0,5):
    print(get_bin_prev("biochemistry_Triglycerides", "cov_DIET", i, 5, data))

1.8613741997736173
1.7682608421793664
1.72152193436322
1.649436796018066
1.5585922531971665


In [21]:
for i in range(0,5):
    print(get_bin_prev("body_WHRadjBMIz", "other_TIME_TV", i, 5, data))

-0.09907324179354242
-0.062371810576635794
-0.03472445533606288
-0.0016038167254468288
0.09357867951922645


In [22]:
print(get_bin_prev("disease_AID_ALL", "other_NO_WHEAT", 0, 2, data))
print(get_bin_prev("disease_AID_ALL", "other_NO_WHEAT", 1, 2, data))

0.11646541022457962
0.30614084507042255


In [23]:
pd.DataFrame({'smoking':data['cov_SMOKING_STATUS'], 'whitecount':data['blood_WHITE_COUNT']}).corr()

,smoking,whitecount
smoking,1.000000,0.190009
whitecount,0.190009,1.000000


In [24]:
data['cov_SMOKING_STATUS']

0         NaN
1         NaN
2         NaN
3         1.0
4         0.0
         ... 
337540    2.0
337541    0.0
337542    0.0
337543    0.0
337544    2.0
Name: cov_SMOKING_STATUS, Length: 337545, dtype: float64

In [9]:
get_Z(get_h2s('disease_T2D', 'cov_ALCOHOL_DESC_ORDER', 5, data))

27.098839588215583